In [1]:
import pandas as pd  # Import pandas for data manipulation and analysis
import numpy as np  # Import numpy for numerical operations
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting data into training and testing sets
from sklearn.preprocessing import StandardScaler, LabelEncoder  # Import StandardScaler for feature scaling and LabelEncoder for encoding target labels
from sklearn.linear_model import LogisticRegression  # Import LogisticRegression for logistic regression model
from sklearn.ensemble import RandomForestClassifier, StackingClassifier  # Import RandomForestClassifier for random forest model and StackingClassifier for model stacking
from sklearn.svm import SVC  # Import SVC for support vector classifier
from xgboost import XGBClassifier  # Import XGBClassifier for XGBoost model
from lightgbm import LGBMClassifier  # Import LGBMClassifier for LightGBM model
from catboost import CatBoostClassifier  # Import CatBoostClassifier for CatBoost model
from sklearn import metrics  # Import metrics from sklearn for model evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score  # Import specific metrics for model evaluation
import warnings  # Import warnings to manage warnings
import category_encoders as ce  # Import category_encoders for encoding categorical features
import optuna
# Ignore warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

In [3]:
path = r'C:\Users\User\Desktop\Rashad\DATA\heart.csv'

# Load dataset from Excel file
df = pd.read_csv(path)

# Set option to display all columns
pd.set_option('display.max_columns', None)

# Display the loaded dataset
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [12]:
print(df.select_dtypes('object').columns)

Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype='object')


In [13]:
models = ["logistic_regression", "random_forest", "svc", "xgboost", "lightgbm", "catboost", "catboost_custom"]

# Create copies of the DataFrame for different preprocessing approaches
df_copy = df.copy()
df_boost = df.copy()
df_cat = df.copy()

# Iterate through each model in the list
for model in models:
    if model == "logistic_regression" or model == "svc":
        # Preprocessing for logistic regression and SVC models
        
        # Fill missing values with mean for numerical columns and mode for categorical columns
        for col in df_copy.columns:
            if df_copy[col].dtype in ['int64', 'float64']:
                df_copy[col] = df_copy[col].fillna(df_copy[col].mean())
            elif df_copy[col].dtype == 'object':
                df_copy[col] = df_copy[col].fillna(df_copy[col].mode().iloc[0])
        
        # Perform Weight of Evidence Encoding on all columns except the target column 'default'
        all_columns = df_copy.columns.tolist()
        all_columns.remove('HeartDisease')
        encoder = ce.WOEEncoder(cols=all_columns)
        df_copy[all_columns] = encoder.fit_transform(df_copy[all_columns], df_copy['HeartDisease'])
        
        # Remove duplicate rows and 'ID' column if exists
        df_copy.drop_duplicates(inplace=True)
        if 'ID' in df_copy.columns:
            df_copy.drop(columns=['ID'], inplace=True)
                
    elif model in ["random_forest", "xgboost", "lightgbm", "catboost"]:
        # Preprocessing for tree-based models
        
        # Perform Weight of Evidence Encoding on all columns except the target column 'default'
        all_columns = df_boost.columns.tolist()
        all_columns.remove('HeartDisease')
        encoder = ce.WOEEncoder(cols=all_columns)
        df_boost[all_columns] = encoder.fit_transform(df_boost[all_columns], df_boost['HeartDisease'])
             
    elif model ==  "catboost_custom":
        # Preprocessing for CatBoost with custom handling of missing values
        
        # Fill missing values in specified categorical columns with 'Missing Value'
        numerical_cols = df_cat.select_dtypes(include=['int64', 'float64']).columns.tolist()
        numerical_cols.remove('HeartDisease')
        columns_to_fill = cat_features=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
        df_cat[columns_to_fill] = df_cat[columns_to_fill].fillna('Missing Value')
        
        # Perform Weight of Evidence Encoding on numerical columns
        encoder = ce.WOEEncoder(cols=numerical_cols)
        df_cat[numerical_cols] = encoder.fit_transform(df_cat[numerical_cols], df_cat['HeartDisease'])

# Split data into features (X) and target (y) for each preprocessing approach
x_log = df_copy.drop(columns=['HeartDisease'])
y_log = df_copy['HeartDisease']
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(x_log, y_log, test_size=0.3, random_state=42)

x_boost = df_boost.drop(columns=['HeartDisease'])
y_boost = df_boost['HeartDisease']
X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(x_boost, y_boost, test_size=0.3, random_state=42)

x_cat = df_cat.drop(columns=['HeartDisease'])
y_cat = df_cat['HeartDisease']
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(x_cat, y_cat, test_size=0.3, random_state=42)

In [18]:
models = []  # Initialize an empty list to store model definitions

# Define default models for each algorithm
xgb_model_def = XGBClassifier()
lgb_model_def = LGBMClassifier()
catboost_model_def = CatBoostClassifier()
catboost_model_custom = CatBoostClassifier(cat_features=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'])
lg = LogisticRegression()
rf = RandomForestClassifier()
svc_model_def = SVC(probability=True)  # Add SVC model definition

# Define models for stacking
stacking_models = [('XGBoost', xgb_model_def),
                   ('LightGBM', lgb_model_def),
                   ('CatBoost', catboost_model_def),
                   ('CatBoost_Custom', catboost_model_custom),
                   ('LogisticRegression', lg),
                   ('RandomForest', rf),
                   ('SVC', svc_model_def)]  # Add SVC to stacking_models

# Extend models list with default models
models.extend([
    ('XGBoost', xgb_model_def),
    ('LightGBM', lgb_model_def),
    ('CatBoost', catboost_model_def),
    ('CatBoost_Custom', catboost_model_custom),
    ('LogisticRegression', lg),
    ('RandomForest', rf),
    ('SVC', svc_model_def)  # Add SVC to models list
])

In [19]:
def train_and_evaluate_model(model_name, model, X_train, y_train, X_test, y_test):
    """
    Train and evaluate the given model on the training and testing data.

    Parameters:
    model_name (str): Name of the model for display purposes.
    model : Machine learning model object.
    X_train : Features of the training data.
    y_train : Target labels of the training data.
    X_test : Features of the testing data.
    y_test : Target labels of the testing data.

    Returns:
    float: Gini coefficient calculated from the model's predictions.
    """

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Predict labels and probabilities on the testing data
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # Calculate ROC AUC and Gini coefficient
    roc_prob = roc_auc_score(y_test, y_prob)
    gini_prob = roc_prob * 2 - 1

    # Calculate confusion matrix and classification report
    confusion_matrix_result = confusion_matrix(y_test, y_pred)
    classification_report_result = classification_report(y_test, y_pred)

    # Print model performance metrics
    print(f'Model Performance for {model_name}')
    print('Gini prob is', gini_prob * 100)
    print(classification_report_result)
    print(confusion_matrix_result)

    return gini_prob

In [20]:
gini_df = pd.DataFrame(columns=['Model', 'Gini_prob'])

# Iterate through each model in the list of models
for model_name, model in models:
    # Train and evaluate the model, and calculate the Gini coefficient
    if model_name == 'CatBoost_Custom':
        gini_prob = train_and_evaluate_model(model_name, model, X_train_cat, y_train_cat, X_test_cat, y_test_cat)
    elif model_name in ['LogisticRegression', 'SVC']:
        gini_prob = train_and_evaluate_model(model_name, model, X_train_log, y_train_log, X_test_log, y_test_log)
    else:
        gini_prob = train_and_evaluate_model(model_name, model, X_train_boost, y_train_boost, X_test_boost, y_test_boost)
        
    # Add model name and Gini coefficient to the DataFrame
    if gini_prob is not None:
        gini_df = pd.concat([gini_df, pd.DataFrame({'Model': [model_name], 'Gini_prob': [gini_prob]})], ignore_index=True)

# Sort the DataFrame by Gini coefficient in descending order
gini_df_sorted = gini_df.sort_values(by='Gini_prob', ascending=False)

Model Performance for XGBoost
Gini prob is 88.61062717770037
              precision    recall  f1-score   support

           0       0.83      0.88      0.86       112
           1       0.92      0.88      0.90       164

    accuracy                           0.88       276
   macro avg       0.87      0.88      0.88       276
weighted avg       0.88      0.88      0.88       276

[[ 99  13]
 [ 20 144]]
[LightGBM] [Info] Number of positive: 344, number of negative: 298
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000093 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 183
[LightGBM] [Info] Number of data points in the train set: 642, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.535826 -> initscore=0.143548
[LightGBM] [Info] Start training from score 0.143548
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

In [21]:
gini_df_sorted

,Model,Gini_prob
4,LogisticRegression,0.944142
3,CatBoost_Custom,0.932491
2,CatBoost,0.929443
6,SVC,0.914307
5,RandomForest,0.910551
1,LightGBM,0.898737
0,XGBoost,0.886106
